In [12]:
import pandas as pd
import numpy as np
import names, json
import random

In [13]:
df = pd.read_csv('data/jcpenney/jcpenney.csv')

In [14]:
df.drop(['sku', 'sale_price', 'category', 'product_url', 'total_number_reviews'], inplace=True, axis=1)
df.rename(columns={'uniq_id': 'id', 'name_title': 'name', 'list_price': 'price', 'average_product_rating': 'rating', 'product_image_urls': 'image', 'Reviews': 'reviews', 'category_tree': 'category'}, inplace=True)

In [15]:
df.drop_duplicates(subset=['name'], inplace=True)
df.dropna(subset=['name'], inplace=True)

In [16]:
df['name'] = df['name'].apply(lambda x: " ".join(str(x).split()))
df['description'] = df['description'].apply(lambda x: " ".join(str(x).split()))
df['description'].fillna("", inplace=True)

In [17]:
# RATING
df['rating'] = df['rating'].str.replace(' out of 5', '')
df['rating'].fillna(np.round(np.random.uniform(0, 5.0), 1), inplace=True)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [18]:
df['category'] = df['category'].apply(lambda x: " ".join(str(x).split()))
df['category'] = df['category'].str.split('|')
df['category'] = df['category'].fillna("").apply(list);

df['stock'] = np.random.randint(1000, size=df.shape[0])
df['store'] = 'jcpenney'
df['price'] = [round(random.uniform(0.01, 300), ndigits=2) for i in range(df.shape[0])]
df['description'].fillna("", inplace=True)

In [19]:
def parseReviews(row):
    try:
        st = row.reviews[13:-3]
        st = st.split('}, {')

        reviews = []
        i = 1
        for s in st:
            string = '"review_' + str(i) + '"=>'
            review = s.replace(string, '')
            review = review[1:-1]
            review = " ".join(review.split())
            i += 1
            aux = {'author': names.get_full_name(), 'rating': np.round(np.random.uniform(0.1, 5.0), 1), 'date': '2018-2-5', 'title': "", 'body': review}
            reviews.append(aux)
        return reviews
    except:
        return []

df['reviews'] = df.apply(parseReviews, axis=1)
df['Id'] = ["products/jcpenney/" + str(x) for x in range(1, len(df) + 1)]
df.drop(columns='id', inplace=True)

In [20]:
df.to_csv('data/jcpenney/products.csv', encoding='utf-8', index=False)

In [21]:
parsed = json.loads(df.to_json(orient="records", default_handler=str))

json_string = json.dumps(parsed, indent=4)
with open('data/output/jcpenney.json', 'w') as outfile:
    outfile.write(json_string)